# Data clean

In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

def procesar_datos_calidad_aire(directorio_archivos, archivo_salida="datos_combinados.xlsx"):
    """
    Procesa múltiples archivos Excel de datos de calidad del aire:
    - Limpia valores erróneos (-99)
    - Combina fecha y hora
    - Une todos los archivos
    - Transforma de formato ancho a largo
    
    Args:
        directorio_archivos (str): Ruta del directorio con los archivos Excel
        archivo_salida (str): Nombre del archivo de salida
    """
    
    # Lista de archivos a procesar
    nombres_archivos = ['NO.xlsx', 'NO2.xlsx', 'NOX.xlsx', 'Ozono.xlsx', 'Temperature.xlsx']
    
    # Diccionario para almacenar los DataFrames procesados
    dataframes_procesados = []
    
    for nombre_archivo in nombres_archivos:
        ruta_archivo = Path(directorio_archivos) / nombre_archivo
        
        if not ruta_archivo.exists():
            print(f"Advertencia: No se encontró el archivo {nombre_archivo}")
            continue
            
        print(f"Procesando {nombre_archivo}...")
        
        try:
            # Leer el archivo Excel
            df = pd.read_excel(ruta_archivo)
            
            # Verificar que existan las columnas necesarias
            if 'FECHA' not in df.columns or 'HORA' not in df.columns:
                print(f"Error: {nombre_archivo} no tiene las columnas FECHA y/o HORA")
                continue
            
            # Limpiar valores erróneos (reemplazar -99 con NaN)
            df_limpio = df.copy()
            
            # Identificar columnas de estaciones (todas excepto FECHA y HORA)
            columnas_estaciones = [col for col in df.columns if col not in ['FECHA', 'HORA']]
            
            # Reemplazar valores erróneos en columnas de estaciones
            for col in columnas_estaciones:
                df_limpio[col] = df_limpio[col].replace(-99, np.nan)
                # También puedes agregar otros valores erróneos si los hay
                df_limpio[col] = df_limpio[col].replace([-999, -9999], np.nan)
            
            # Convertir FECHA a datetime si no lo está
            df_limpio['FECHA'] = pd.to_datetime(df_limpio['FECHA'], format='%d/%m/%Y')
            
            # Crear datetime combinando fecha y hora
            # Asumiendo que HORA va de 1 a 24, donde 24 = 00:00 del día siguiente
            df_limpio['HORA_AJUSTADA'] = df_limpio['HORA'].copy()
            df_limpio.loc[df_limpio['HORA'] == 1, 'HORA_AJUSTADA'] = 0
            
            # Crear la columna datetime
            df_limpio['FECHA_HORA'] = df_limpio['FECHA'] + pd.to_timedelta(df_limpio['HORA_AJUSTADA'], unit='h')
            
            
            # Transformar a formato largo
            # Primero, seleccionar las columnas relevantes
            columnas_para_melt = ['FECHA_HORA'] + columnas_estaciones
            df_subset = df_limpio[columnas_para_melt].copy()
            
            # Convertir a formato largo
            df_largo = pd.melt(
                df_subset,
                id_vars=['FECHA_HORA'],
                value_vars=columnas_estaciones,
                var_name='ESTACION',
                value_name='VALOR'
            )
            
            # Agregar columna con el tipo de contaminante
            contaminante = nombre_archivo.replace('.xlsx', '').upper()
            df_largo['CONTAMINANTE'] = contaminante
            
            # Eliminar filas con valores NaN si lo deseas
            # df_largo = df_largo.dropna(subset=['VALOR'])
            
            dataframes_procesados.append(df_largo)
            
            print(f"✓ {nombre_archivo} procesado exitosamente")
            print(f"  - Filas originales: {len(df)}")
            print(f"  - Filas en formato largo: {len(df_largo)}")
            print(f"  - Estaciones encontradas: {len(columnas_estaciones)}")
            
        except Exception as e:
            print(f"Error procesando {nombre_archivo}: {str(e)}")
    
    # Combinar todos los DataFrames
    if dataframes_procesados:
        df_combinado_largo = pd.concat(dataframes_procesados, ignore_index=True)
        
        # Convertir a formato ancho con cada contaminante como columna
        print("Convirtiendo a formato ancho con contaminantes como columnas...")
        
        df_final = df_combinado_largo.pivot_table(
            index=['FECHA_HORA', 'ESTACION'],
            columns='CONTAMINANTE',
            values='VALOR',
            aggfunc='first'  # En caso de duplicados, toma el primero
        ).reset_index()
        
        # Aplanar los nombres de columnas
        df_final.columns.name = None
        
        # Reordenar columnas para que sea más legible
        columnas_base = ['FECHA_HORA', 'ESTACION']
        columnas_contaminantes = [col for col in df_final.columns if col not in columnas_base]
        columnas_contaminantes.sort()  # Ordenar alfabéticamente
        
        df_final = df_final[columnas_base + columnas_contaminantes]
        
        # Ordenar por fecha y estación
        df_final = df_final.sort_values(['FECHA_HORA', 'ESTACION'])
        
        # Resetear índice
        df_final = df_final.reset_index(drop=True)
        
        # Guardar el archivo final
        ruta_salida = Path(directorio_archivos) / archivo_salida
        df_final.to_excel(ruta_salida, index=False)
        
        print(f"\n✓ Procesamiento completado!")
        print(f"✓ Archivo guardado como: {ruta_salida}")
        print(f"✓ Total de registros: {len(df_final):,}")
        print(f"✓ Rango de fechas: {df_final['FECHA_HORA'].min()} a {df_final['FECHA_HORA'].max()}")
        print(f"✓ Contaminantes (columnas): {', '.join(columnas_contaminantes)}")
        print(f"✓ Estaciones: {', '.join(df_final['ESTACION'].unique())}")
        
        return df_final
    else:
        print("No se procesaron archivos exitosamente.")
        return None

def calcular_media_movil_8h(df):
    """
    Calcula la media móvil de 8 horas para todos los contaminantes
    y selecciona el máximo diario de ozono por estación
    
    Args:
        df: DataFrame con los datos en formato ancho (contaminantes como columnas)
        
    Returns:
        tuple: (df_media_movil, df_ozono_max_diario)
    """
    if df is None:
        return None, None
    
    print("\nCalculando medias móviles de 8 horas...")
    
    # Identificar columnas de contaminantes (todas excepto FECHA_HORA y ESTACION)
    columnas_contaminantes = [col for col in df.columns if col not in ['FECHA_HORA', 'ESTACION']]
    print(f"Contaminantes encontrados: {', '.join(columnas_contaminantes)}")
    
    # Crear lista para almacenar resultados
    resultados_media_movil = []
    resultados_ozono_max = []
    
    # Procesar cada estación por separado
    for estacion in df['ESTACION'].unique():
        print(f"Procesando estación: {estacion}")
        
        # Filtrar datos para esta estación
        subset = df[df['ESTACION'] == estacion].copy()
        
        if len(subset) == 0:
            continue
        
        # Ordenar por fecha y hora
        subset = subset.sort_values('FECHA_HORA')
        subset = subset.reset_index(drop=True)
        
        # Calcular medias móviles para cada contaminante
        for contaminante in columnas_contaminantes:
            if contaminante in subset.columns:
                # Crear nombre de columna para la media móvil
                nombre_media = f"{contaminante}_MEDIA_8H"
                
                # Calcular media móvil de 8 horas
                subset[nombre_media] = subset[contaminante].rolling(
                    window=8, 
                    min_periods=6,  # Al menos 6 valores válidos de 8 (75%)
                    center=False
                ).mean()
        
        # Agregar información de ventana para el registro
        subset['HORA_INICIO_VENTANA'] = subset['FECHA_HORA']
        subset['HORA_FIN_VENTANA'] = subset['FECHA_HORA'] + pd.Timedelta(hours=7)
        
        # Solo conservar registros que tengan al menos una media móvil válida
        columnas_medias = [col for col in subset.columns if col.endswith('_MEDIA_8H')]
        
        # Crear una máscara para registros con al menos una media válida
        mask_valido = subset[columnas_medias].notna().any(axis=1)
        subset_valido = subset[mask_valido].copy()
        
        if len(subset_valido) > 0:
            resultados_media_movil.append(subset_valido)
            
            # Para ozono, calcular máximo diario si existe
            columna_ozono = None
            for col in columnas_contaminantes:
                if 'OZONO' in col.upper():
                    columna_ozono = col
                    break
            
            if columna_ozono is not None and f"{columna_ozono}_MEDIA_8H" in subset_valido.columns:
                # Agregar columna de fecha (sin hora)
                subset_valido['FECHA'] = subset_valido['FECHA_HORA'].dt.date
                
                # Encontrar el máximo diario por estación
                ozono_col_media = f"{columna_ozono}_MEDIA_8H"
                subset_ozono = subset_valido.dropna(subset=[ozono_col_media])
                
                if len(subset_ozono) > 0:
                    ozono_max_diario = subset_ozono.loc[
                        subset_ozono.groupby('FECHA')[ozono_col_media].idxmax()
                    ].copy()
                    
                    # Renombrar la columna de ozono para simplicidad
                    ozono_max_diario = ozono_max_diario.rename(columns={ozono_col_media: 'OZONO_MAXIMO_8H'})
                    
                    resultados_ozono_max.append(ozono_max_diario)
    
    # Combinar todos los resultados
    df_media_movil = None
    df_ozono_max_diario = None
    
    if resultados_media_movil:
        df_media_movil = pd.concat(resultados_media_movil, ignore_index=True)
        df_media_movil = df_media_movil.sort_values(['ESTACION', 'FECHA_HORA'])
        
        print(f"✓ Medias móviles calculadas: {len(df_media_movil):,} registros")
        
        # Mostrar columnas de medias móviles creadas
        columnas_medias_creadas = [col for col in df_media_movil.columns if col.endswith('_MEDIA_8H')]
        print(f"✓ Columnas de medias móviles: {', '.join(columnas_medias_creadas)}")
    
    if resultados_ozono_max:
        df_ozono_max_diario = pd.concat(resultados_ozono_max, ignore_index=True)
        df_ozono_max_diario = df_ozono_max_diario.sort_values(['ESTACION', 'FECHA'])
        
        print(f"✓ Máximos diarios de ozono: {len(df_ozono_max_diario):,} registros")
    
    return df_media_movil, df_ozono_max_diario

def mostrar_resumen_datos(df, df_media_movil=None, df_ozono_max=None):
    """
    Muestra un resumen estadístico de los datos procesados
    """
    if df is None:
        return
    
    print("\n" + "="*60)
    print("RESUMEN DE DATOS")
    print("="*60)
    
    # Identificar columnas de contaminantes
    columnas_contaminantes = [col for col in df.columns if col not in ['FECHA_HORA', 'ESTACION']]
    
    # Resumen datos originales
    print("\nDATOS ORIGINALES:")
    print(f"Total de registros: {len(df):,}")
    print(f"Contaminantes (columnas): {', '.join(columnas_contaminantes)}")
    
    print("\nConteo por estación:")
    print(df['ESTACION'].value_counts())
    
    # Estadísticas por contaminante
    print("\nEstadísticas por contaminante:")
    for contaminante in columnas_contaminantes:
        if contaminante in df.columns:
            valores = df[contaminante]
            print(f"\n{contaminante}:")
            print(f"  - Registros totales: {len(valores):,}")
            print(f"  - Registros válidos: {valores.count():,}")
            print(f"  - Valores faltantes: {valores.isna().sum():,}")
            if valores.count() > 0:
                print(f"  - Promedio: {valores.mean():.2f}")
                print(f"  - Mínimo: {valores.min():.2f}")
                print(f"  - Máximo: {valores.max():.2f}")
    
    # Resumen medias móviles
    if df_media_movil is not None:
        print(f"\nMEDIAS MÓVILES 8H:")
        print(f"Total de registros: {len(df_media_movil):,}")
        
        # Identificar columnas de medias móviles
        columnas_medias = [col for col in df_media_movil.columns if col.endswith('_MEDIA_8H')]
        print(f"Medias móviles calculadas: {', '.join(columnas_medias)}")
        
        print("\nConteo por estación (medias móviles):")
        print(df_media_movil['ESTACION'].value_counts())
    
    # Resumen ozono máximo diario
    if df_ozono_max is not None:
        print(f"\nOZONO MÁXIMO DIARIO:")
        print(f"Total de registros: {len(df_ozono_max):,}")
        print("Conteo por estación:")
        print(df_ozono_max['ESTACION'].value_counts())
        
        # Estadísticas de ozono
        if 'OZONO_MAXIMO_8H' in df_ozono_max.columns:
            ozono_stats = df_ozono_max['OZONO_MAXIMO_8H'].describe()
            print(f"\nEstadísticas de ozono máximo diario:")
            print(f"  - Promedio: {ozono_stats['mean']:.2f}")
            print(f"  - Mediana: {ozono_stats['50%']:.2f}")
            print(f"  - Mínimo: {ozono_stats['min']:.2f}")
            print(f"  - Máximo: {ozono_stats['max']:.2f}")
    
    print("\n" + "="*60)

# Ejemplo de uso
if __name__ == "__main__":
    # Especifica la ruta donde están tus archivos Excel
    directorio_archivos = "."  # Cambia esto por tu ruta
    
    # Procesar los datos básicos
    datos_procesados = procesar_datos_calidad_aire(
        directorio_archivos=directorio_archivos,
        archivo_salida="datos_calidad_aire_procesados.xlsx"
    )
    
    if datos_procesados is not None:
        # Calcular medias móviles de 8 horas
        datos_media_movil, ozono_max_diario = calcular_media_movil_8h(datos_procesados)
        
        # Guardar archivos adicionales
        if datos_media_movil is not None:
            ruta_media_movil = Path(directorio_archivos) / "datos_media_movil_8h.xlsx"
            datos_media_movil.to_excel(ruta_media_movil, index=False)
            print(f"✓ Medias móviles guardadas en: {ruta_media_movil}")
            
            # También en CSV
            ruta_media_movil_csv = Path(directorio_archivos) / "datos_media_movil_8h.csv"
            datos_media_movil.to_csv(ruta_media_movil_csv, index=False)
        
        if ozono_max_diario is not None:
            ruta_ozono_max = Path(directorio_archivos) / "ozono_maximo_diario.xlsx"
            ozono_max_diario.to_excel(ruta_ozono_max, index=False)
            print(f"✓ Ozono máximo diario guardado en: {ruta_ozono_max}")
            
            # También en CSV
            ruta_ozono_max_csv = Path(directorio_archivos) / "ozono_maximo_diario.csv"
            ozono_max_diario.to_csv(ruta_ozono_max_csv, index=False)
        
        # Mostrar resumen completo
        mostrar_resumen_datos(datos_procesados, datos_media_movil, ozono_max_diario)
        
        # Guardar datos originales también como CSV
        ruta_csv = Path(directorio_archivos) / "datos_calidad_aire_procesados.csv"
        datos_procesados.to_csv(ruta_csv, index=False)
        print(f"\n✓ Datos originales también guardados como CSV: {ruta_csv}")

Procesando NO.xlsx...
✓ NO.xlsx procesado exitosamente
  - Filas originales: 8776
  - Filas en formato largo: 307160
  - Estaciones encontradas: 35
Procesando NO2.xlsx...
✓ NO2.xlsx procesado exitosamente
  - Filas originales: 8776
  - Filas en formato largo: 315936
  - Estaciones encontradas: 36
Procesando NOX.xlsx...
✓ NOX.xlsx procesado exitosamente
  - Filas originales: 8776
  - Filas en formato largo: 307160
  - Estaciones encontradas: 35
Procesando Ozono.xlsx...
✓ Ozono.xlsx procesado exitosamente
  - Filas originales: 8776
  - Filas en formato largo: 315936
  - Estaciones encontradas: 36
Procesando Temperature.xlsx...
✓ Temperature.xlsx procesado exitosamente
  - Filas originales: 8776
  - Filas en formato largo: 245728
  - Estaciones encontradas: 28
Convirtiendo a formato ancho con contaminantes como columnas...

✓ Procesamiento completado!
✓ Archivo guardado como: datos_calidad_aire_procesados.xlsx
✓ Total de registros: 240,684
✓ Rango de fechas: 2024-01-01 00:00:00 a 2025-01-